In [1]:
]activate @covid19

 Activating new environment at `~/.julia/environments/covid19/Project.toml`


In [2]:
using Pkg
Pkg.update()

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Updating `~/.julia/environments/covid19/Project.toml`
 [no changes]
   Updating `~/.julia/environments/covid19/Manifest.toml`
 [no changes]


In [3]:
#]add CSV HTTP GitHub DataFrames PyCall PyPlot Plots Dates Statistics TimeSeries MLJ XGBoost Measurements LsqFit NCDatasets Glob JSON3 Tables Pandas FileIO CSVFiles Missings JLD2 OnlineStats CUDAnative CuArrays
ENV["PYTHON"] = ENV["CONDA_PYTHON_EXE"]
#Pkg.build("PyCall")

"/bluedrive/anaconda3/bin/python"

In [4]:
dir_covid19 = "/bluedrive/GitHub/covid19"
cd(dir_covid19)
if (dir_covid19 in LOAD_PATH) == false
    push!(LOAD_PATH, dir_covid19);
end

4-element Array{String,1}:
 "@"
 "@v#.#"
 "@stdlib"
 "/bluedrive/GitHub/covid19"

In [5]:
using Conda, PyCall, CSVFiles, DataFrames

In [ ]:
# data source: https://github.com/nytimes/covid-19-data
cv19nyt_county = DataFrame(load("https://github.com/nytimes/covid-19-data/raw/master/us-counties.csv"));
cv19nyt_state = DataFrame(load("https://github.com/nytimes/covid-19-data/raw/master/us-states.csv"));

# data source: https://github.com/CSSEGISandData/COVID-19
fipslookup = DataFrame(load("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv"));
ts_cv19_confirm_global = DataFrame(load("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv");
ts_cv19_confirm_US = DataFrame(load("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv");
ts_cv19_death_global = DataFrame(load("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv");
ts_cv19_death_US = DataFrame(load("https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv");


,date,county,state,fips,cases,deaths
,Dates…,String,String,Int64⍰,Int64,Int64
1,2020-01-21,Snohomish,Washington,53061,1,0
2,2020-01-22,Snohomish,Washington,53061,1,0
3,2020-01-23,Snohomish,Washington,53061,1,0
4,2020-01-24,Cook,Illinois,17031,1,0
5,2020-01-24,Snohomish,Washington,53061,1,0
6,2020-01-25,Orange,California,6059,1,0
7,2020-01-25,Cook,Illinois,17031,1,0
8,2020-01-25,Snohomish,Washington,53061,1,0
9,2020-01-26,Maricopa,Arizona,4013,1,0


In [1]:
module COVID19
# This code loads COVID19 data from https://github.com/CSSEGISandData/COVID-19
#
# COVID19:
# https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
#
# FIPS: 
# https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv

using CSV, HTTP, DataFrames, Dates

export confirmed, death, recovered, t, lat, lon, country, region

function load_covid19()
    covid19url_confirmed_US = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
    covid19url_confirmed_global = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
    covid19url_death_US = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
    covid19url_death_global = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
    covid19url_recovered_global = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
    covid19url_fips = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv"

    covid19path_confirmed_US = ENV["HOME"] * "/Box/Data/COVID19/covid19_confirmed_US.csv"
    covid19path_confirmed_global = ENV["HOME"] * "/Box/Data/COVID19/covid19_confirmed_global.csv"
    covid19path_death_US = ENV["HOME"] * "/Box/Data/COVID19/covid19_death_US.csv"
    covid19path_death_global = ENV["HOME"] * "/Box/Data/COVID19/covid19_death_global.csv"
    covid19path_recovered_global = ENV["HOME"] * "/Box/Data/COVID19/covid19_recovered_global.csv"
    covid19path_fips = ENV["HOME"] * "/Box/Data/COVID19/UID_ISO_FIPS_LookUp_Table.csv"

    download(covid19url_confirmed_US, covid19path_confirmed_US)
    download(covid19url_confirmed_global, covid19path_confirmed_global)
    download(covid19url_death_US, covid19path_death_US)
    download(covid19url_death_global, covid19path_death_global)
    download(covid19url_recovered_global, covid19path_recovered_global)
    download(covid19url_fips, covid19path_fips)

#=     io = open(covid19path_confirmed, "w")
    r = HTTP.request("GET", covid19url_confirmed, response_stream=io)
    close(io)
    io = open(covid19path_death, "w")
    r = HTTP.request("GET", covid19url_death, response_stream=io)
    close(io)
    io = open(covid19path_recovered, "w")
    r = HTTP.request("GET", covid19url_recovered, response_stream=io)
    close(io) =#

    confirmed_US = CSV.read(covid19path_confirmed_US);
    death_US = CSV.read(covid19path_death_US);
    confirmed_global = CSV.read(covid19path_confirmed_global);
    death_global = CSV.read(covid19path_death_global);
    recovered_global = CSV.read(covid19path_recovered_global);
    fipstable = CSV.read(covid19path_fips);

    return confirmed_US, death_US, confirmed_global, death_global, recovered_global, fipstable
end
(confirmed_US,death_US, confirmed_global, death_global, recovered_global, fipstable) = load_covid19()

t0 = Date("2020-01-22")
ndays = Dates.today() - t0;
t = collect(t0 : Day(1) : Dates.today());

c0_global = 5;
c0_US = 12;

latg = confirmed_global.Lat;
long = confirmed_global.Long;
country = confirmed_global[:,2];
region = confirmed_global[:,3];
fips = fipstable;


end


┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1260


ArgumentError: ArgumentError: Package HTTP not found in current path:
- Run `import Pkg; Pkg.add("HTTP")` to install the HTTP package.
